In [1]:
##### import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import torch.nn.functional as F
import cv2
import numpy as np
import torch.utils.data as Data
from PIL import  Image,ImageDraw
import numpy as np
import random
import torch 

Batch_Size = 50
num_epochs = 200
learning_rate = 1e-5
samples = 20000

data1 = []
data2= []


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% --  Generate dataset


cap = cv2.VideoCapture('video_drone_tank.avi')

uframes = []
ret, frame = cap.read()

uframes.append(frame)
flag = 0
z = 0
while(z<30):
    ret, frame = cap.read()
    #np_frame = cv2.imread('video', frame) # does not work
    #np_frame = np.asarray(cv2.GetMat(frame)) # does not work
    #print(np_frame.shape)
    for a in uframes:
        if (np.array(a).any() != None):
            print("a: ",np.shape(a))
            print("frame: ",np.shape(a))
            difference = cv2.subtract(np.array(a), np.array(frame))    
            result = not np.any(difference)
            #print("inside for")
            if result is True:
                flag = 1
                #print("setting flag")
                break
    if(not flag):
        #print("appending")
        uframes.append(frame)
    z+=1

for i,frame in enumerate(uframes):
    uframes[i] = cv2.resize(frame,(100,100))
    
print(np.shape(uframes[0]))
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
print("length: ",len(uframes))
for i in range(samples):
    switch = random.randint(0,len(uframes)-1)
    data1.append(np.array(np.transpose(np.asarray(uframes[switch]),(2,0,1)),dtype=np.float32))
    data2.append(np.array(np.transpose(np.asarray(uframes[switch]),(2,0,1)),dtype=np.float32))

    

#%%%%%%%%%%%%%%%%%%%%%%%%%%% Noisy data

im1 = Image.open('tree1.png')
im1 = im1.resize((10,10))

im2 = Image.open('tree2.png')
im2 = im2.resize((10,10))

im3 = Image.open('tree3.png')
im3 = im3.resize((10,10))

img = Image.new('RGB', (100,100), color = '#9b7653')
img.paste(im1,(5,75),im1)
img.paste(im1,(10,55),im1)
img.paste(im1,(25,50),im1)
img.paste(im2,(75,5),im2)
img.paste(im2,(60,30),im2)
img.paste(im2,(90,20),im2)
img.paste(im3,(75,60),im3)
img.paste(im3,(60,80),im3)
im_n = np.reshape(img,(100,100,3))
b = np.asarray(im_n)
b = np.array(b, dtype=np.float32)
b = np.transpose(b,(2,0,1))
b = torch.from_numpy(b)

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%       

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Data Preparation

y0 = np.asarray(data1)
y1 = np.asarray(data2)



print(np.shape(y0))

x = torch.from_numpy(y0)
y = torch.from_numpy(y1)


torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=Batch_Size,

    shuffle=True,

    num_workers=0,

)


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Defining NN

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()

        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()

        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(20000, 15)
        self.sigmoid1 = nn.Sigmoid()

    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)

        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)

        # Max pool 2 
        out = self.maxpool2(out)

        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        #out = self.sigmoid1(out)
        return out
    
model = CNNModel()


optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Loss Function

def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)
    #print(fx, np.shape(fx))
    #print(gy, np.shape(gy))
    #print(np.shape((torch.transpose(fx,3,2))),np.shape(fx))
    covf = torch.matmul(fx.t(), fx) / Nsamples

    covg = torch.matmul(gy.t(), gy) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-4)





for epoch in range(num_epochs):
    for x,y in loader:
        optimizer.zero_grad()
        # ===================forward=====================
        #loss = criterion(output1, img)
        loss = h_score(model(x),model(y))
        # ===================backward====================

        #optimizer_1.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer_1.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))
    print("x: ", model(x[0].reshape(1,3,100, 100)))
    print("y:", model(y[0].reshape(1,3,100, 100)))
    print("b:", model(b.reshape(1,3,100, 100)))


a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 

epoch [3/200], loss:-6.3214
x:  tensor([[ 46.4939,  22.9433, -27.4100, -21.9092,   4.9229,  -3.1195,  -2.1120,
          -8.5231,  34.2540,  -3.9164,   0.4376,  -5.2446,  15.6974,   3.6571,
          -3.2494]], grad_fn=<AddmmBackward>)
y: tensor([[ 46.4939,  22.9433, -27.4100, -21.9092,   4.9229,  -3.1195,  -2.1120,
          -8.5231,  34.2540,  -3.9164,   0.4376,  -5.2446,  15.6974,   3.6571,
          -3.2494]], grad_fn=<AddmmBackward>)
b: tensor([[ 36.4167,  15.0413, -19.0730, -18.5715,   1.0111,  -2.9029,  -2.7848,
          -7.9626,  25.3450,  -0.8319,   1.8930,  -5.7082,  15.0193,   0.9870,
          -5.0645]], grad_fn=<AddmmBackward>)
epoch [4/200], loss:-6.3444
x:  tensor([[ 35.3238,  35.7211,  -1.9929,  -7.0030,  11.0324, -28.2911, -30.4820,
          -3.7446,  49.8288,  -1.5619,   7.4230,   6.2069,   1.7836,  -8.6776,
          -2.0834]], grad_fn=<AddmmBackward>)
y: tensor([[ 35.3238,  35.7211,  -1.9929,  -7.0030,  11.0324, -28.2911, -30.4820,
          -3.7446,  49.8288,  -1

epoch [16/200], loss:-6.5967
x:  tensor([[ 10.2877,  11.0499,  -3.3713, -10.9658,  12.4183, -10.8440,   3.4741,
          -0.6824,  25.8136,   3.8758,  25.0008, -12.6894,  10.8944,   8.6001,
          36.4738]], grad_fn=<AddmmBackward>)
y: tensor([[ 10.2877,  11.0499,  -3.3713, -10.9658,  12.4183, -10.8440,   3.4741,
          -0.6824,  25.8136,   3.8758,  25.0008, -12.6894,  10.8944,   8.6001,
          36.4738]], grad_fn=<AddmmBackward>)
b: tensor([[  9.0272,   6.8939,  -1.5601,  -8.3334,   7.7474,  -8.5662,   1.1836,
          -2.4622,  18.7818,   3.7331,  20.3276, -11.7706,   8.9289,   4.4194,
          25.1711]], grad_fn=<AddmmBackward>)
epoch [17/200], loss:-7.4359
x:  tensor([[ -0.1434,  18.3774,   8.5987,   2.8967,  31.4317, -13.8679, -10.1890,
           1.6550,  26.2581,   1.9605,  16.7168,   2.9622,  30.4435, -13.8893,
           1.3273]], grad_fn=<AddmmBackward>)
y: tensor([[ -0.1434,  18.3774,   8.5987,   2.8967,  31.4317, -13.8679, -10.1890,
           1.6550,  26.2581,  

epoch [29/200], loss:-6.9535
x:  tensor([[ -3.4936,   6.2514,  -5.6952,  -1.8210,  22.1923,  -4.5187, -22.1349,
          -6.1175,   6.3559,  -7.8466,   6.4530, -16.9172,  -3.0565,   0.5366,
          32.7166]], grad_fn=<AddmmBackward>)
y: tensor([[ -3.4936,   6.2514,  -5.6952,  -1.8210,  22.1923,  -4.5187, -22.1349,
          -6.1175,   6.3559,  -7.8466,   6.4530, -16.9172,  -3.0565,   0.5366,
          32.7166]], grad_fn=<AddmmBackward>)
b: tensor([[ -1.9288,   3.8252,  -3.6701,  -1.1588,  16.3815,  -5.2775, -17.5183,
          -7.7650,   3.3220,  -5.6242,   4.4499, -14.6120,  -1.4304,  -2.0067,
          23.8130]], grad_fn=<AddmmBackward>)
epoch [30/200], loss:-7.2070
x:  tensor([[ -1.5533,   3.1783, -10.7777,  -1.6316,  17.7153,  -6.0135,  -9.1332,
          -7.8587,   4.0149,  -7.8599,  15.3211, -18.4136,   5.8325,  -1.2881,
           0.1835]], grad_fn=<AddmmBackward>)
y: tensor([[ -1.5533,   3.1783, -10.7777,  -1.6316,  17.7153,  -6.0135,  -9.1332,
          -7.8587,   4.0149,  

epoch [42/200], loss:-6.9640
x:  tensor([[ -0.6169,   3.2483,   0.2606,  -6.4153,   3.7010,   4.3956,  -3.7590,
           0.4922,   2.4607, -12.4409,   6.0434,  17.9788,  -0.1066,  17.4734,
          -8.5394]], grad_fn=<AddmmBackward>)
y: tensor([[ -0.6169,   3.2483,   0.2606,  -6.4153,   3.7010,   4.3956,  -3.7590,
           0.4922,   2.4607, -12.4409,   6.0434,  17.9788,  -0.1066,  17.4734,
          -8.5394]], grad_fn=<AddmmBackward>)
b: tensor([[ 1.6369,  1.4362,  1.5784, -5.0155,  2.3279,  1.9950, -2.2187, -3.1112,
          1.3227, -8.8703,  4.2453, 10.8772,  1.1695, 12.3147, -8.6477]],
       grad_fn=<AddmmBackward>)
epoch [43/200], loss:-6.9223
x:  tensor([[  1.0127,   9.1317,  -1.8313,   0.4555,   8.5123,  -4.9528,   2.2087,
          -4.4363,  -2.5340,  -4.9910,  22.0938,  13.4257,  -0.5206,   2.0238,
         -11.7117]], grad_fn=<AddmmBackward>)
y: tensor([[  1.0127,   9.1317,  -1.8313,   0.4555,   8.5123,  -4.9528,   2.2087,
          -4.4363,  -2.5340,  -4.9910,  22.0938

...

epoch [137/200], loss:-6.6514
x:  tensor([[ 7.1377,  1.7226,  0.1553,  2.3012, -7.6106, -8.2519, -5.8608, -1.9619,
          4.2554, -8.1348, -3.4372,  2.7809, -6.0156, -3.1607, -0.6795]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 7.1377,  1.7226,  0.1553,  2.3012, -7.6106, -8.2519, -5.8608, -1.9619,
          4.2554, -8.1348, -3.4372,  2.7809, -6.0156, -3.1607, -0.6795]],
       grad_fn=<AddmmBackward>)
b: tensor([[ 6.2171,  0.2553,  0.3973,  1.1250, -6.2212, -5.9954, -3.8712, -3.1473,
          2.8232, -5.7605, -0.8198, -0.2632, -2.9475, -2.7695, -2.8547]],
       grad_fn=<AddmmBackward>)
epoch [138/200], loss:-7.2283
x:  tensor([[  6.0284,   2.1389,  -3.3049,  -0.8178,   2.0332,  -6.5150,  -8.2873,
          -2.2897,  12.2728,  -4.6946,  -0.6501, -12.9104,  -3.4909,   3.9401,
          -0.2497]], grad_fn=<AddmmBackward>)
y: tensor([[  6.0284,   2.1389,  -3.3049,  -0.8178,   2.0332,  -6.5150,  -8.2873,
          -2.2897,  12.2728,  -4.6946,  -0.6501, -12.9104,  -3.4909,   3.9401,


epoch [150/200], loss:-7.1301
x:  tensor([[ 0.2443,  2.0919, -2.4694, -1.0026,  4.8048, -3.7362, -7.6464, -2.3337,
         15.6953, -4.1831,  2.1896, -4.5030,  3.5513, -2.6962,  7.0682]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 0.2443,  2.0919, -2.4694, -1.0026,  4.8048, -3.7362, -7.6464, -2.3337,
         15.6953, -4.1831,  2.1896, -4.5030,  3.5513, -2.6962,  7.0682]],
       grad_fn=<AddmmBackward>)
b: tensor([[-0.1466,  0.3336, -1.4344, -1.8479,  2.3489, -2.6083, -4.9261, -3.3141,
         10.6648, -3.3361,  2.4928, -5.1347,  5.5060, -1.1758,  3.0869]],
       grad_fn=<AddmmBackward>)
epoch [151/200], loss:-7.4129
x:  tensor([[ 3.1985,  0.7956, -4.7398,  0.4558,  0.3709, -4.5845, -1.9196, -3.0563,
         17.9296, -0.7126,  3.8543, -4.0496, 10.0041,  5.3499,  4.7292]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 3.1985,  0.7956, -4.7398,  0.4558,  0.3709, -4.5845, -1.9196, -3.0563,
         17.9296, -0.7126,  3.8543, -4.0496, 10.0041,  5.3499,  4.7292]],
       grad_fn=<Addm

epoch [164/200], loss:-6.8266
x:  tensor([[ 5.4705,  1.8995, -3.0170,  1.0002, 14.6934, -4.5480,  2.7499, -0.6685,
          9.8556, -2.2312,  4.0477, -1.5898,  4.7476,  2.5069,  6.6999]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 5.4705,  1.8995, -3.0170,  1.0002, 14.6934, -4.5480,  2.7499, -0.6685,
          9.8556, -2.2312,  4.0477, -1.5898,  4.7476,  2.5069,  6.6999]],
       grad_fn=<AddmmBackward>)
b: tensor([[ 2.5931,  0.3863, -1.1892, -0.1651,  8.4313, -3.3348,  0.7536, -2.1940,
          5.7353, -1.1469,  3.5406, -2.6994,  5.4886,  1.2140,  2.2724]],
       grad_fn=<AddmmBackward>)
epoch [165/200], loss:-7.3520
x:  tensor([[ 7.4489,  1.6472, -5.1943,  0.2399, -0.4165, -3.1888,  3.1240, -1.8670,
          0.6014, -1.8478,  0.0175,  2.1789,  6.2802, -4.8107,  4.3392]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 7.4489,  1.6472, -5.1943,  0.2399, -0.4165, -3.1888,  3.1240, -1.8670,
          0.6014, -1.8478,  0.0175,  2.1789,  6.2802, -4.8107,  4.3392]],
       grad_fn=<Addm

epoch [178/200], loss:-6.9615
x:  tensor([[-1.3308,  0.0878, -3.4080,  0.4595,  2.0394, -4.1704, -6.9786, -0.9336,
         10.0920, -4.6846,  3.6007,  7.3533, -0.8594, -5.5074, -1.2704]],
       grad_fn=<AddmmBackward>)
y: tensor([[-1.3308,  0.0878, -3.4080,  0.4595,  2.0394, -4.1704, -6.9786, -0.9336,
         10.0920, -4.6846,  3.6007,  7.3533, -0.8594, -5.5074, -1.2704]],
       grad_fn=<AddmmBackward>)
b: tensor([[-0.9223, -0.9194, -2.0652, -0.7194,  0.7434, -3.0420, -7.1471, -2.3253,
          6.7016, -3.5375,  3.3874,  2.6859, -0.0893, -3.6856, -1.4804]],
       grad_fn=<AddmmBackward>)
epoch [179/200], loss:-7.1609
x:  tensor([[ 4.3919,  2.1486, -9.4542, -0.2654,  7.1737, -2.6819,  1.3454,  0.0401,
         10.3310,  0.7721,  0.5791, -7.3223,  5.8566,  4.5792,  0.5645]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 4.3919,  2.1486, -9.4542, -0.2654,  7.1737, -2.6819,  1.3454,  0.0401,
         10.3310,  0.7721,  0.5791, -7.3223,  5.8566,  4.5792,  0.5645]],
       grad_fn=<Addm

epoch [191/200], loss:-6.8697
x:  tensor([[-3.7236, -3.3494, -0.5757, -0.1516,  4.6246,  0.9310, -6.6002, -2.9334,
         16.2948,  1.8690,  1.2789,  2.7549,  2.5449, -1.8694, -1.9626]],
       grad_fn=<AddmmBackward>)
y: tensor([[-3.7236, -3.3494, -0.5757, -0.1516,  4.6246,  0.9310, -6.6002, -2.9334,
         16.2948,  1.8690,  1.2789,  2.7549,  2.5449, -1.8694, -1.9626]],
       grad_fn=<AddmmBackward>)
b: tensor([[-2.1483, -3.0980,  0.3333, -1.1827,  3.0242,  0.1541, -5.5034, -3.6792,
          9.9555,  0.2535,  1.6613, -0.0509,  3.1881, -2.4447, -1.7258]],
       grad_fn=<AddmmBackward>)
epoch [192/200], loss:-6.7699
x:  tensor([[ 4.5858,  1.6203, -4.5682, -0.1302,  8.8580, -5.4913, -7.5679, -1.9781,
         14.8925, -1.6399,  3.9200, -4.1762,  9.2259, -5.4498,  0.5286]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 4.5858,  1.6203, -4.5682, -0.1302,  8.8580, -5.4913, -7.5679, -1.9781,
         14.8925, -1.6399,  3.9200, -4.1762,  9.2259, -5.4498,  0.5286]],
       grad_fn=<Addm

In [3]:
data3=[]

class_data = []

for i in range(75):
    img = uframes[0]
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    #draw = ImageDraw.Draw(img)
    #draw.ellipse((50,50,55,55), fill=(0, 0, 255))
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),0))



In [4]:
data3=[]

for i in range(75):
    img = uframes[1]

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),2))
    
print(len(class_data))

150


In [5]:
data3=[]

for i in range(75):
    img = uframes[2]
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),3))
    
print(len(class_data))

225


In [41]:
import random
p=0
class_data = []
while(p<len(uframes)):
    data3=[]
    print(p)
    for i in range(2):
        img = uframes[p]
        data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
        y3 = np.array(data3)
        x_1 = torch.from_numpy(y3)
        temp = model(x_1[i].reshape(1,3,100,100))
        class_data.append((list(temp.detach().numpy()[0]),p))
    p+=1
    print(len(class_data))

random.shuffle(class_data)
random.shuffle(class_data)

0
2
1
4
2
6
3
8
4
10
5
12
6
14
7
16
8
18
9
20
10
22
11
24


In [6]:
data3=[]

for i in range(75):
    img = uframes[3]

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),1))
    
print(len(class_data))

300


In [18]:

cap = cv2.VideoCapture('video_drone_tank.avi')

uframes = []
ret, frame = cap.read()

uframes.append(frame)
flag = 0
z = 0
while(z<30):
    ret, frame = cap.read()
    #np_frame = cv2.imread('video', frame) # does not work
    #np_frame = np.asarray(cv2.GetMat(frame)) # does not work
    #print(np_frame.shape)
    for a in uframes:
        if (np.array(a).any() != None):
            print("a: ",np.shape(a))
            print("frame: ",np.shape(a))
            difference = cv2.subtract(np.array(a), np.array(frame))    
            result = not np.any(difference)
            #print("inside for")
            if result is True:
                flag = 1
                #print("setting flag")
                break
    if(not flag):
        #print("appending")
        uframes.append(frame)
    z+=1

frame = Image.fromarray(uframes[0])


video = cv2.VideoWriter('video_test.avi', 0, 1, (720,1080))

for image in uframes:
    video.write(image)


for i,frame in enumerate(uframes):
    uframes[i] = cv2.resize(frame,(100,100))
cv2.destroyAllWindows()
video.release()

a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 

In [43]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from colorama import init 
from termcolor import colored 
import time

# import some data to play with

# Take the first two features. We could avoid this by using a two-dim dataset
X = [(i[0]) for i in class_data]
y = [i[1] for i in class_data]
#print(X,np.shape(X))
#print(y,np.shape(y))
# we create an instance of SVM and fit out data. We do not scale our
# data since we want to plot the support vectors
C = 1.0  # SVM regularization parameter
clf = svm.SVC(kernel='linear', C=C)
clf = clf.fit(X, y)
data3=[]
y3=0

n1 = "\x1b[31m\"Drone Stationary, Tank Firing\"\x1b[0m"
n2 = "\x1b[31m\"Drone Reloading, Tank Firing\"\x1b[0m"
n3 = "\x1b[31m\"Drone Firing, Tank Stationary\"\x1b[0m"
n4 = "\x1b[31m\"Drone Firing, Tank Reloading\"\x1b[0m"
n5 = "\x1b[31m\"Drone Firing, Tank Firing\"\x1b[0m"

names = ['Just Background','Only Tank moving', 'Only Drone Moving', 'Drone stationary, Tank Stationary', 'Drone Stationary, Tank Reloading' ,
         n1, 'Drone reloading, Tank Stationary', 'Drone reloading, Tank Reloading', n2 , n3,n4,n5]
time.sleep(5)
print("Start")
for i in range(12):
    time.sleep(1)
    img = uframes[0]
    ms = Image.fromarray(img)
    ms.show()
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))
    temp = temp.detach().numpy()
    #print(temp)
    z = clf.predict(temp)
    print(names[z[0]])
      
    

Start
Just Background
Just Background
Just Background
Just Background
Just Background
Just Background
Just Background
Just Background
Just Background
Just Background
Just Background
Just Background


In [ ]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    #draw = ImageDraw.Draw(img)
    #draw.ellipse((50,50,55,55), fill=(0, 0, 255))
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

In [ ]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    img.paste(im_drone,(5,5),im_drone)
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

In [ ]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    img.paste(im_drone,(72,65),im_drone)
    img.paste(imb,(80,80),imb)
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

In [1]:
x = [1,2,3,4,5]

len(x)

5